In [1]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy
import UA

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./kit.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Kit_Info'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_User_Agent())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service(str_chromedriver), options=option)

                browser.maximize_window()
                
                # = = = = = = = = = = = = = = =
                
                browser.get(url)

                time.sleep(3)
                
                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@aria-label, "More Information for")]')))
                list_oe = browser.find_elements(by=By.XPATH, value='//section[contains(@aria-label, "Alternate/OEM Part Number")]')

                oe = ''
                if len(list_oe) != 0:
                    for _ in list_oe:
                        if _.text.startswith('{Alternate Inventory Numbers: '):
                            oe += _.text.replace('{Alternate Inventory Numbers: ', '').replace('}', '') + ';'
                        elif _.text.startswith('OEM / Interchange Numbers: '):
                            oe += _.text.replace('OEM / Interchange Numbers: ', '') + ';'

                oe = ';'.join(oe[:-1].split(', '))

                # = = = = = = = = = = = = = = =
                
                src = browser.find_elements(by=By.XPATH, value='//img[@id="inlineimg[1]"]')
                if len(src) == 0:
                    src = ''
                else:
                    src = src[0].get_attribute('src')
                    
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Kit_Info': url,
                                         'Kit_OE': oe,
                                         'Kit_Src': src}])
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                browser.quit()
                time.sleep(3)
                continue
                
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Kit_Info': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./kit_oe_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：2710

https://www.rockauto.com/en/moreinfo.php?pk=1572494&cc=0&pt=8616  <->  [ok] - 剩余数量：2695
https://www.rockauto.com/en/moreinfo.php?pk=1572374&cc=0&pt=5552  <->  [ok] - 剩余数量：2694
https://www.rockauto.com/en/moreinfo.php?pk=315442&cc=0&pt=8616  <->  [ok] - 剩余数量：2693
https://www.rockauto.com/en/moreinfo.php?pk=315447&cc=0&pt=5552  <->  [ok] - 剩余数量：2692
https://www.rockauto.com/en/moreinfo.php?pk=1572452&cc=0&pt=5556  <->  [ok] - 剩余数量：2691
https://www.rockauto.com/en/moreinfo.php?pk=315215&cc=0&pt=8616  <->  [ok] - 剩余数量：2690
https://www.rockauto.com/en/moreinfo.php?pk=315363&cc=0&pt=8616  <->  [ok] - 剩余数量：2689
https://www.rockauto.com/en/moreinfo.php?pk=1572514&cc=0&pt=5552  <->  [ok] - 剩余数量：2688
https://www.rockauto.com/en/moreinfo.php?pk=1572514&cc=0&pt=5552  <->  [ok] - 剩余数量：2687
https://www.rockauto.com/en/moreinfo.php?pk=6851444&cc=0&pt=5552  <->  [ok] - 剩余数量：2686
https://www.rockauto.com/en/moreinfo.php?pk=6851444&cc=0&pt=5552  <->  [ok] - 剩余数量：2685
https://www.rockauto.com/e

https://www.rockauto.com/en/moreinfo.php?pk=5044515&cc=0&pt=5552  <->  [ok] - 剩余数量：2601
https://www.rockauto.com/en/moreinfo.php?pk=8502264&cc=0&pt=8616  <->  [ok] - 剩余数量：2600
https://www.rockauto.com/en/moreinfo.php?pk=6338703&cc=0&pt=8616  <->  [ok] - 剩余数量：2599
https://www.rockauto.com/en/moreinfo.php?pk=7671440&cc=0&pt=5552  <->  [ok] - 剩余数量：2598
https://www.rockauto.com/en/moreinfo.php?pk=5760090&cc=0&pt=8616  <->  [ok] - 剩余数量：2597
https://www.rockauto.com/en/moreinfo.php?pk=315391&cc=0&pt=8616  <->  [ok] - 剩余数量：2596
https://www.rockauto.com/en/moreinfo.php?pk=5044515&cc=0&pt=5552  <->  [ok] - 剩余数量：2595
https://www.rockauto.com/en/moreinfo.php?pk=315215&cc=0&pt=8616  <->  [ok] - 剩余数量：2594
https://www.rockauto.com/en/moreinfo.php?pk=6338709&cc=0&pt=5552  <->  [ok] - 剩余数量：2593
https://www.rockauto.com/en/moreinfo.php?pk=3997389&cc=0&pt=5552  <->  [ok] - 剩余数量：2592
https://www.rockauto.com/en/moreinfo.php?pk=5044518&cc=0&pt=5552  <->  [ok] - 剩余数量：2591
https://www.rockauto.com/en/morein

https://www.rockauto.com/en/moreinfo.php?pk=4663836&cc=0&pt=8616  <->  [ok] - 剩余数量：2507
https://www.rockauto.com/en/moreinfo.php?pk=5760024&cc=0&pt=8616  <->  [ok] - 剩余数量：2506
https://www.rockauto.com/en/moreinfo.php?pk=10998700&cc=0&pt=5556  <->  [ok] - 剩余数量：2505
https://www.rockauto.com/en/moreinfo.php?pk=315216&cc=0&pt=8616  <->  [ok] - 剩余数量：2504
https://www.rockauto.com/en/moreinfo.php?pk=4663983&cc=0&pt=5556  <->  [ok] - 剩余数量：2503
https://www.rockauto.com/en/moreinfo.php?pk=8103976&cc=0&pt=5552  <->  [ok] - 剩余数量：2502
https://www.rockauto.com/en/moreinfo.php?pk=3997251&cc=0&pt=5552  <->  [ok] - 剩余数量：2501
https://www.rockauto.com/en/moreinfo.php?pk=1015606&cc=0&pt=5552  <->  [ok] - 剩余数量：2500
https://www.rockauto.com/en/moreinfo.php?pk=1015555&cc=0&pt=5552  <->  [ok] - 剩余数量：2499
https://www.rockauto.com/en/moreinfo.php?pk=315218&cc=0&pt=5552  <->  [ok] - 剩余数量：2498
https://www.rockauto.com/en/moreinfo.php?pk=315403&cc=0&pt=5552  <->  [ok] - 剩余数量：2497
https://www.rockauto.com/en/morein

https://www.rockauto.com/en/moreinfo.php?pk=489220&cc=0&pt=5552  <->  [ok] - 剩余数量：2413
https://www.rockauto.com/en/moreinfo.php?pk=489143&cc=0&pt=5552  <->  [ok] - 剩余数量：2412
https://www.rockauto.com/en/moreinfo.php?pk=315423&cc=0&pt=8616  <->  [ok] - 剩余数量：2411
https://www.rockauto.com/en/moreinfo.php?pk=315384&cc=0&pt=5552  <->  [ok] - 剩余数量：2410
https://www.rockauto.com/en/moreinfo.php?pk=7671552&cc=0&pt=5552  <->  [ok] - 剩余数量：2409
https://www.rockauto.com/en/moreinfo.php?pk=7671556&cc=0&pt=8616  <->  [ok] - 剩余数量：2408
https://www.rockauto.com/en/moreinfo.php?pk=7671816&cc=0&pt=8616  <->  [ok] - 剩余数量：2407
https://www.rockauto.com/en/moreinfo.php?pk=7671820&cc=0&pt=5552  <->  [ok] - 剩余数量：2406
https://www.rockauto.com/en/moreinfo.php?pk=7671472&cc=0&pt=8616  <->  [ok] - 剩余数量：2405
https://www.rockauto.com/en/moreinfo.php?pk=6851464&cc=0&pt=5552  <->  [ok] - 剩余数量：2404
https://www.rockauto.com/en/moreinfo.php?pk=6851464&cc=0&pt=5552  <->  [ok] - 剩余数量：2403
https://www.rockauto.com/en/moreinfo

https://www.rockauto.com/en/moreinfo.php?pk=12396481&cc=0&pt=5552  <->  [ok] - 剩余数量：2318
https://www.rockauto.com/en/moreinfo.php?pk=5044554&cc=0&pt=5552  <->  [ok] - 剩余数量：2317
https://www.rockauto.com/en/moreinfo.php?pk=13016433&cc=0&pt=8616  <->  [ok] - 剩余数量：2316
https://www.rockauto.com/en/moreinfo.php?pk=3997281&cc=0&pt=5552  <->  [ok] - 剩余数量：2315
https://www.rockauto.com/en/moreinfo.php?pk=7671596&cc=0&pt=5556  <->  [ok] - 剩余数量：2314
https://www.rockauto.com/en/moreinfo.php?pk=9095516&cc=0&pt=5552  <->  [ok] - 剩余数量：2313
https://www.rockauto.com/en/moreinfo.php?pk=12396489&cc=0&pt=8616  <->  [ok] - 剩余数量：2312
https://www.rockauto.com/en/moreinfo.php?pk=7671588&cc=0&pt=8616  <->  [ok] - 剩余数量：2311
https://www.rockauto.com/en/moreinfo.php?pk=5044551&cc=0&pt=5552  <->  [ok] - 剩余数量：2310
https://www.rockauto.com/en/moreinfo.php?pk=9095512&cc=0&pt=5552  <->  [ok] - 剩余数量：2309
https://www.rockauto.com/en/moreinfo.php?pk=4663980&cc=0&pt=5552  <->  [ok] - 剩余数量：2308
https://www.rockauto.com/en/m

...

https://www.rockauto.com/en/moreinfo.php?pk=13016309&cc=0&pt=5552  <->  [ok] - 剩余数量：339
https://www.rockauto.com/en/moreinfo.php?pk=13016421&cc=0&pt=5552  <->  [ok] - 剩余数量：338
https://www.rockauto.com/en/moreinfo.php?pk=13825625&cc=0&pt=8616  <->  [ok] - 剩余数量：337
https://www.rockauto.com/en/moreinfo.php?pk=13016285&cc=0&pt=5552  <->  [ok] - 剩余数量：336
https://www.rockauto.com/en/moreinfo.php?pk=13016425&cc=0&pt=5552  <->  [ok] - 剩余数量：335
https://www.rockauto.com/en/moreinfo.php?pk=13016261&cc=0&pt=5552  <->  [ok] - 剩余数量：334
https://www.rockauto.com/en/moreinfo.php?pk=10998624&cc=0&pt=8616  <->  [ok] - 剩余数量：333
https://www.rockauto.com/en/moreinfo.php?pk=13016525&cc=0&pt=5552  <->  [ok] - 剩余数量：332
https://www.rockauto.com/en/moreinfo.php?pk=7671980&cc=0&pt=5552  <->  [ok] - 剩余数量：331
https://www.rockauto.com/en/moreinfo.php?pk=13825589&cc=0&pt=5552  <->  [ok] - 剩余数量：330
https://www.rockauto.com/en/moreinfo.php?pk=3997272&cc=0&pt=8616  <->  [ok] - 剩余数量：329
https://www.rockauto.com/en/morein

https://www.rockauto.com/en/moreinfo.php?pk=14834461&cc=0&pt=5552  <->  [ok] - 剩余数量：245
https://www.rockauto.com/en/moreinfo.php?pk=14834409&cc=0&pt=5552  <->  [ok] - 剩余数量：244
https://www.rockauto.com/en/moreinfo.php?pk=7671616&cc=0&pt=8616  <->  [ok] - 剩余数量：243
https://www.rockauto.com/en/moreinfo.php?pk=13828861&cc=0&pt=5552  <->  [ok] - 剩余数量：242
https://www.rockauto.com/en/moreinfo.php?pk=7671620&cc=0&pt=5552  <->  [ok] - 剩余数量：241
https://www.rockauto.com/en/moreinfo.php?pk=10998756&cc=0&pt=5552  <->  [ok] - 剩余数量：240
https://www.rockauto.com/en/moreinfo.php?pk=14834449&cc=0&pt=8616  <->  [ok] - 剩余数量：239
https://www.rockauto.com/en/moreinfo.php?pk=7671584&cc=0&pt=5556  <->  [ok] - 剩余数量：238
https://www.rockauto.com/en/moreinfo.php?pk=14834409&cc=0&pt=5552  <->  [ok] - 剩余数量：237
https://www.rockauto.com/en/moreinfo.php?pk=9095532&cc=0&pt=8616  <->  [ok] - 剩余数量：236
https://www.rockauto.com/en/moreinfo.php?pk=14834525&cc=0&pt=5552  <->  [ok] - 剩余数量：235
https://www.rockauto.com/en/moreinfo

https://www.rockauto.com/en/moreinfo.php?pk=13828909&cc=0&pt=5552  <->  [ok] - 剩余数量：150
https://www.rockauto.com/en/moreinfo.php?pk=13828865&cc=0&pt=8616  <->  [ok] - 剩余数量：149
https://www.rockauto.com/en/moreinfo.php?pk=13828845&cc=0&pt=5552  <->  [ok] - 剩余数量：148
https://www.rockauto.com/en/moreinfo.php?pk=13828933&cc=0&pt=8616  <->  [ok] - 剩余数量：147
https://www.rockauto.com/en/moreinfo.php?pk=14834457&cc=0&pt=5552  <->  [ok] - 剩余数量：146
https://www.rockauto.com/en/moreinfo.php?pk=14834513&cc=0&pt=5556  <->  [ok] - 剩余数量：145
https://www.rockauto.com/en/moreinfo.php?pk=12396585&cc=0&pt=5552  <->  [ok] - 剩余数量：144
https://www.rockauto.com/en/moreinfo.php?pk=14044601&cc=0&pt=8616  <->  [ok] - 剩余数量：143
https://www.rockauto.com/en/moreinfo.php?pk=10998728&cc=0&pt=8616  <->  [ok] - 剩余数量：142
https://www.rockauto.com/en/moreinfo.php?pk=7671452&cc=0&pt=5556  <->  [ok] - 剩余数量：141
https://www.rockauto.com/en/moreinfo.php?pk=13828877&cc=0&pt=5552  <->  [ok] - 剩余数量：140
https://www.rockauto.com/en/morei

https://www.rockauto.com/en/moreinfo.php?pk=3997350&cc=0&pt=8616  <->  [ok] - 剩余数量：55
https://www.rockauto.com/en/moreinfo.php?pk=7671856&cc=0&pt=5552  <->  [ok] - 剩余数量：54
https://www.rockauto.com/en/moreinfo.php?pk=315418&cc=0&pt=5556  <->  [ok] - 剩余数量：53
https://www.rockauto.com/en/moreinfo.php?pk=315282&cc=0&pt=5324  <->  [ok] - 剩余数量：52
https://www.rockauto.com/en/moreinfo.php?pk=489293&cc=0&pt=5552  <->  [ok] - 剩余数量：51
https://www.rockauto.com/en/moreinfo.php?pk=1313567&cc=0&pt=5552  <->  [ok] - 剩余数量：50
https://www.rockauto.com/en/moreinfo.php?pk=3997350&cc=0&pt=8616  <->  [ok] - 剩余数量：49
https://www.rockauto.com/en/moreinfo.php?pk=6851496&cc=0&pt=8616  <->  [ok] - 剩余数量：48
https://www.rockauto.com/en/moreinfo.php?pk=6851484&cc=0&pt=5552  <->  [ok] - 剩余数量：47
https://www.rockauto.com/en/moreinfo.php?pk=1572414&cc=0&pt=5556  <->  [ok] - 剩余数量：46
https://www.rockauto.com/en/moreinfo.php?pk=3997248&cc=0&pt=8756  <->  [ok] - 剩余数量：45
https://www.rockauto.com/en/moreinfo.php?pk=6851540&cc=0&

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
